<div style="background-color:AliceBlue; padding:10px; border-left:5px solid #6495ed; margin-bottom:10px;">
  <h4 style="font-size:13px;">📦 <strong>Importar Librerias</strong></h4>
</div>

In [1]:
import sys
import ray
from ray import serve
import mlflow
import pandas as pd
from fastapi import FastAPI
from fastapi import Request
from pydantic import BaseModel, ConfigDict
from fastapi.exceptions import RequestValidationError
from fastapi.responses import JSONResponse
from starlette.requests import Request
from json import JSONDecodeError
from sklearn.datasets import load_iris
#import requests

<div style="background-color:AliceBlue; padding:10px; border-left:5px solid #2e8b57; margin-bottom:10px;">
  <h4 style="font-size:13px;">🤖 <strong>Configuracion del Modelo</strong></h4>
</div>

In [2]:
try:
    class Item(BaseModel):
        model_config = ConfigDict(extra="forbid")
        c1: float
        c2: float
        c3: float
        c4: float

    app = FastAPI(
        title="Import Predictor API",
        description="Import Pipeline online inference",
    )

    @app.exception_handler(RequestValidationError)
    async def validation_exception_handler(request: Request, exc: RequestValidationError):
        mensajes = [err["msg"] for err in exc.errors()]
        return JSONResponse(
            status_code=422,
            content={
                "cod": 422,
                "msg": "; ".join(mensajes),
                "data": None,
                "errors": exc.errors(),
            },
        )

    @serve.deployment(ray_actor_options={"num_cpus": 0.1})
    @serve.ingress(app)
    class ModelMlFlow:
        def __init__(self, model_uri: str):
            mlflow.set_tracking_uri("http://127.0.0.1:8080")
            try:
                self.model = mlflow.pyfunc.load_model(model_uri)
                #self.model = mlflow.sklearn.load_model(model_uri)
            except Exception as ex:
                print(f"MlFlow: Se produjo el siguiente error : {ex} ")

        @app.post("/")
        async def score(self, item: dict):  # <- evitar pickling de 'Item'
            try:
                #payload = item
                #df = pd.DataFrame([payload])
                #score = self.model.predict(df)
                #score_value = float(score.iloc[0])
                df = pd.DataFrame([item])
                score_value = float(self.model.predict(df).iat[0])
                return {
                    "cod": 200,
                    "msg": "Score calculated successfully.",
                    "data": {"em_eventprobability": score_value},
                    "errors": None,
                }
            except Exception as e:
                return {
                    "cod": 500,
                    "msg": "Error calculating score",
                    "data": None,
                    "errors": str(e),
                }

except Exception as ex:
    print(f"Desployment y Ingress: Se produjo el siguiente error : {ex} ")

<div style="background-color:AliceBlue; padding:10px; border-left:5px solid #2e8b57; margin-bottom:10px;">
  <h4 style="font-size:13px;">🤖 <strong>Conecta a un cluster de Ray ya iniciado</strong></h4>
</div>

In [3]:
try:
    ray.init(address="ray://127.0.0.1:10001")
    serve.start(detached=True, http_options={"host": "0.0.0.0", "port": 8000})
    print("✅ Conectado a Ray existente.")
except Exception as e:
    sys.exit(f"❌ No se pudo conectar a ray://127.0.0.1:10001 → {e}")

2025-08-21 20:52:16,219	INFO client_builder.py:244 -- Passing the following kwargs to ray.init() on the server: log_to_driver
SIGTERM handler is not set because current thread is not the main thread.
INFO 2025-08-21 20:52:20,309 serve 18912 -- Connecting to existing Serve app in namespace "serve". New http options will not be applied.


✅ Conectado a Ray existente.


<div style="background-color:AliceBlue; padding:10px; border-left:5px solid #2e8b57; margin-bottom:10px;">
  <h4 style="font-size:13px;">🤖 <strong>Desplegar el Modelo</strong></h4>
</div>

In [4]:
MODEL_URI = "models:/model-pyfunc/1"   # o "models:/iris-models/Production"
NAME = "model-pyfunc"
ROUTE_PREFIX = "/v1/model-pyfunc"

graph = ModelMlFlow.bind(MODEL_URI)

try:
    serve.run(graph, name=NAME, route_prefix=ROUTE_PREFIX)
    print(f"✅ Listo en http://127.0.0.1:8000{ROUTE_PREFIX}")
except Exception as ex:
    print(f"Desployment y Ingress: Se produjo el siguiente error : {ex}")
    try:
        serve.delete(NAME)
        print(f"🧹 App '{NAME}' eliminada por fallo de deploy.")
    except Exception as cleanup_ex:
        print(f"⚠️ No se pudo eliminar '{NAME}': {cleanup_ex}")

INFO 2025-08-21 20:52:20,321 serve 18912 -- Connecting to existing Serve app in namespace "serve". New http options will not be applied.
WARNING 2025-08-21 20:52:20,321 serve 18912 -- The new client HTTP config differs from the existing one in the following fields: ['host', 'location']. The new HTTP config is ignored.
INFO 2025-08-21 20:52:23,391 serve 18912 -- Application 'model-pyfunc' is ready at http://0.0.0.0:8000/v1/model-pyfunc.
INFO 2025-08-21 20:52:23,391 serve 18912 -- Deployed app 'model-pyfunc' successfully.


✅ Listo en http://127.0.0.1:8000/v1/model-pyfunc


In [5]:
ray.shutdown()